# Traindata 전처리

# JSON -> DataFrame

In [ ]:
import os
import json
import pandas as pd

# 라벨링 JSON 파일이 있는 최상위 폴더 경로
label_root = "/media/usou/PortableSSD/mldl/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training/라벨링데이터/"

# 실제 WAV 파일이 존재하는 원천 데이터의 최상위 경로
wav_root = "/media/usou/PortableSSD/mldl/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training/원천데이터/"

# 정상적으로 처리된 데이터 정보를 담을 리스트
data = []

# 오류 발생 시 해당 JSON 파일 또는 존재하지 않는 WAV 경로를 저장할 리스트
broken_files = []

# 라벨링 폴더 내부의 모든 JSON 파일을 재귀적으로 탐색
for folder_path, _, files in os.walk(label_root):
    for file_name in files:
        if file_name.endswith(".json"):
            # 현재 JSON 파일의 전체 경로 구성
            json_path = os.path.join(folder_path, file_name)
            try:
                # JSON 파일 열기 및 파싱
                with open(json_path, 'r', encoding='utf-8') as f:
                    content = json.load(f)

                # JSON 내부 정보 추출
                emotion = content["화자정보"]["Emotion"]
                style = content["화자정보"].get("SpeechStyle", "N/A")
                sensitivity = content["화자정보"].get("Sensitivity", "N/A")
                wav_file = content["파일정보"]["FileName"]

                # 현재 JSON 경로를 라벨 기준 상대경로로 변환
                relative_path = os.path.relpath(folder_path, start=label_root)

                # 상대 경로에서 모든 TL을 TS로 변경
                relative_path = relative_path.replace("TL", "TS")

                # WAV 경로를 원천 데이터 기준으로 재구성
                wav_path = os.path.join(wav_root, relative_path, wav_file)

                # WAV 파일 존재 여부 확인
                if os.path.exists(wav_path):
                    # 정상 데이터 추가
                    data.append({
                        "wav_path": wav_path,
                        "emotion": emotion,
                        "style": style,
                        "sensitivity": sensitivity
                    })
                else:
                    # WAV 파일이 존재하지 않는 경우 로그에 기록
                    print(f"WAV 파일 없음: {wav_path}")
                    broken_files.append(wav_path)

            except Exception as e:
                # JSON 파싱 중 오류 발생 시 기록
                print(f"JSON 읽기 오류: {json_path}: {e}")
                broken_files.append(json_path)

# 정상적으로 수집된 데이터를 DataFrame으로 변환
df = pd.DataFrame(data)

# 결과 CSV 파일로 저장
os.makedirs("./data/usou", exist_ok=True)
df.to_csv("./data/usou/metadata_cleaned.csv", index=False)

# 오류가 발생한 경로들을 텍스트 파일로 저장
with open("./data/usou/broken_files.txt", "w") as f:
    for path in broken_files:
        f.write(path + "\n")

# 최종 처리 결과 출력
print(f"정상 처리된 JSON 수: {len(df)}")
print(f"에러 발생 수: {len(broken_files)}")


# MFCC 추출
-  MFCC 추출이란
    - 음성에서 특징을 뽑아낸 백터
-  데이터 형태
    - 2차원 배열(시간 프레임수, 13)
- 배치
    - 배치 : 전체 데이터를 나누어 처리하는 단위
- 나누는 이유
    - 메모리 부족으로 컴퓨터 프리징 발생





In [ ]:
import os
import librosa
import pandas as pd
import numpy as np
from tqdm import tqdm

# ============================
# 1. 메타데이터 로드
# ============================
# 사전에 정제된 메타데이터 CSV 파일 경로
csv_path = "/media/usou/PortableSSD/mldl_project/data/metadata_cleaned.csv"
df = pd.read_csv(csv_path)

# ============================
# 2. 설정값 정의
# ============================
sample_rate = 16000            # 음성 파일 샘플링 레이트 (Hz)
max_duration = 5.0             # WAV 파일 최대 로딩 시간 (초) → 너무 긴 파일 방지
save_interval = 10000          # 몇 개마다 배치로 저장할지 설정

# 저장용 리스트 초기화
mfcc_features = []             # MFCC 벡터 리스트
labels = []                    # 감정 레이블 리스트
error_files = []               # 처리 중 실패한 파일 목록
save_counter = 0               # 배치 저장 인덱스

# 저장 디렉토리 설정
save_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches"
os.makedirs(save_dir, exist_ok=True)

# ============================
# 3. MFCC 추출 루프
# ============================
for idx, row in tqdm(df.iterrows(), total=len(df)):
    wav_path = row["wav_path"]  # 메타데이터에 포함된 wav 파일 전체 경로
    try:
        # WAV 파일 로딩 (최대 max_duration 초까지만 로드)
        y, sr = librosa.load(wav_path, sr=sample_rate, duration=max_duration)

        # MFCC 13차원 추출
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

        # 시간 축 기준으로 전치 (time_step, n_mfcc)
        mfcc_features.append(mfcc.T)
        labels.append(row["emotion"])

    except Exception as e:
        # 에러 발생 시 파일 경로 저장
        print(f"Error processing {wav_path}: {e}")
        error_files.append(wav_path)

    # 일정 수 이상 쌓이면 배치 저장 후 메모리 초기화
    if len(mfcc_features) >= save_interval:
        np.save(os.path.join(save_dir, f"mfcc_batch_{save_counter}.npy"), np.array(mfcc_features, dtype=object))
        np.save(os.path.join(save_dir, f"label_batch_{save_counter}.npy"), np.array(labels))
        save_counter += 1
        mfcc_features = []
        labels = []

# 남은 데이터가 있다면 마지막 배치 저장
if mfcc_features:
    np.save(os.path.join(save_dir, f"mfcc_batch_{save_counter}.npy"), np.array(mfcc_features, dtype=object))
    np.save(os.path.join(save_dir, f"label_batch_{save_counter}.npy"), np.array(labels))

# ============================
# 4. 에러 파일 저장
# ============================
error_log_path = "/media/usou/PortableSSD/mldl_project/data/broken_audio_files.txt"
with open(error_log_path, "w") as f:
    for path in error_files:
        f.write(path + "\n")

# ============================
# 5. 처리 결과 출력
# ============================
print(f"성공적으로 저장된 배치 수: {save_counter + 1}")
print(f"실패한 파일 수: {len(error_files)}")


# 레이블 인코딩

In [ ]:
import os
import numpy as np
import glob
import pickle
from sklearn.preprocessing import LabelEncoder

# ============================
# 1. 설정
# ============================
# 레이블 배치가 저장된 경로
label_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches"

# 인코딩된 레이블 저장 경로
encoded_label_dir = os.path.join(label_dir, "encoded_labels")
os.makedirs(encoded_label_dir, exist_ok=True)

# ============================
# 2. 모든 배치 레이블 수집
# ============================
# label_batch_*.npy 파일 경로 리스트
label_files = sorted(glob.glob(os.path.join(label_dir, "label_batch_*.npy")))

# 전체 레이블 리스트 생성
all_labels = []
batch_label_data = []  # 배치별 데이터도 임시 저장
for label_file in label_files:
    labels = np.load(label_file, allow_pickle=True)
    batch_label_data.append(labels)
    all_labels.extend(labels)

# ============================
# 3. 레이블 인코딩
# ============================
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

# 인코더 저장 (추후 예측 결과 복원용)
with open(os.path.join(label_dir, "label_encoder.pkl"), "wb") as f:
    pickle.dump(label_encoder, f)

# ============================
# 4. 인코딩된 레이블 배치별로 저장
# ============================
for i, labels in enumerate(batch_label_data):
    encoded = label_encoder.transform(labels)
    save_path = os.path.join(encoded_label_dir, f"label_batch_{i}.npy")
    np.save(save_path, encoded)

print(f"총 레이블 개수: {len(all_labels)}")
print(f"인코딩된 클래스 목록: {label_encoder.classes_}")
print(f"배치 수: {len(label_files)}")
print("레이블 인코딩 및 저장 완료")


# CNN 모델 정의

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_cnn_model(input_shape, num_classes):
    """
    CNN 기반 음성 감정 분류 모델 정의

    Parameters:
        input_shape (tuple): 입력 데이터 형태 (예: (시간축 길이, MFCC 차원 수, 채널 수))
        num_classes (int): 분류할 감정 클래스 수

    Returns:
        tensorflow.keras.Model: 컴파일 완료된 CNN 모델
    """
    model = models.Sequential()

    # 첫 번째 컨볼루션 레이어: 필터 수 32, 커널 사이즈 3x3, 활성화 함수 ReLU
    model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    # 배치 정규화: 학습 안정성과 속도 개선
    model.add(layers.BatchNormalization())
    # 최대 풀링: 출력 크기 절반으로 줄임 (특징 추출과 과적합 방지)
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # 두 번째 컨볼루션 레이어: 필터 수 64
    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # 세 번째 컨볼루션 레이어: 필터 수 128
    model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    # 전역 평균 풀링: 전체 피처 맵의 평균을 계산하여 1D 벡터로 변환
    model.add(layers.GlobalAveragePooling2D())

    # 완전 연결층(Dense Layer) 추가
    model.add(layers.Dense(128, activation='relu'))
    # 과적합 방지를 위한 드롭아웃 (30%)
    model.add(layers.Dropout(0.3))
    # 출력층: softmax로 감정 클래스 확률 예측
    model.add(layers.Dense(num_classes, activation='softmax'))

    # 모델 컴파일: Adam 옵티마이저, sparse_categorical_crossentropy 손실 함수, 정확도 지표
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model


# Validation 전처리

# 메타데이터 csv로 저장

In [ ]:
import os
import json
import pandas as pd

# ========================================
# 1. 경로 설정
# ========================================

# 라벨링 JSON 파일이 저장된 루트 폴더
label_root = "/media/usou/PortableSSD/mldl/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/라벨링데이터/VL1"

# 실제 음성 WAV 파일이 있는 루트 폴더
wav_root = "/media/usou/PortableSSD/mldl/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/VS1"

# ========================================
# 2. 결과 저장 리스트 초기화
# ========================================
data = []             # 메타데이터 저장용 리스트
broken_files = []     # 에러 발생한 파일 로그용 리스트

# ========================================
# 3. JSON 파일 순회 및 정보 추출
# ========================================
for folder_path, _, files in os.walk(label_root):
    for file_name in files:
        if file_name.endswith(".json"):
            json_path = os.path.join(folder_path, file_name)
            try:
                # JSON 파일 열기
                with open(json_path, 'r', encoding='utf-8') as f:
                    content = json.load(f)

                # 화자 정보에서 감정, 스타일, 세부 감정 추출
                emotion = content["화자정보"]["Emotion"]
                style = content["화자정보"].get("SpeechStyle", "N/A")
                sensitivity = content["화자정보"].get("Sensitivity", "N/A")

                # WAV 파일 이름 추출
                wav_file = content["파일정보"]["FileName"]

                # 현재 JSON 경로에서 라벨 루트를 기준으로 상대 경로 추출
                relative_path = os.path.relpath(folder_path, start=label_root)

                # 실제 WAV 파일 경로 생성
                wav_path = os.path.join(wav_root, relative_path, wav_file)

                # WAV 파일이 존재하면 메타데이터에 추가
                if os.path.exists(wav_path):
                    data.append({
                        "wav_path": wav_path,
                        "emotion": emotion,
                        "style": style,
                        "sensitivity": sensitivity
                    })
                else:
                    # WAV 파일이 없는 경우 기록
                    print(f"WAV 파일 없음: {wav_path}")
                    broken_files.append(wav_path)

            except Exception as e:
                # JSON 파싱 실패 시 기록
                print(f"JSON 읽기 오류: {json_path}, 에러: {e}")
                broken_files.append(json_path)

# ========================================
# 4. 결과 저장
# ========================================

# DataFrame 생성
df = pd.DataFrame(data)

# 저장 경로 생성
os.makedirs("/media/usou/PortableSSD/mldl_project/data/validation", exist_ok=True)

# 메타데이터 CSV 저장
df.to_csv("/media/usou/PortableSSD/mldl_project/data/validation/metadata_cleaned_val.csv", index=False)

# 에러 파일 로그 저장
with open("/media/usou/PortableSSD/mldl_project/data/validation/broken_val_files.txt", "w") as f:
    for path in broken_files:
        f.write(path + "\n")

# 요약 출력
print(f"정상 처리된 JSON 수: {len(df)}")
print(f"에러 발생 수: {len(broken_files)}")


# MFCC 추출 Validation 용

In [ ]:
import os
import librosa
import pandas as pd
import numpy as np
from tqdm import tqdm

# ========================================
# 1. 메타데이터 로드
# ========================================

# validation용 정제된 메타데이터 CSV 경로
csv_path = "/media/usou/PortableSSD/mldl_project/data/validation/metadata_cleaned_val.csv"
df = pd.read_csv(csv_path)

# ========================================
# 2. 설정값 정의
# ========================================

sample_rate = 16000             # 음성 샘플링 레이트 (16kHz)
max_duration = 5.0              # WAV 최대 로딩 시간 (초)
save_interval = 10000           # 배치 저장 기준 개수
save_dir = "/media/usou/PortableSSD/mldl_project/data/validation/mfcc_batches"
os.makedirs(save_dir, exist_ok=True)

# 저장용 리스트 초기화
mfcc_features = []              # 추출된 MFCC 벡터 리스트
labels = []                     # 감정 레이블 리스트
error_files = []                # 실패한 파일 목록
save_counter = 0                # 배치 파일 번호

# ========================================
# 3. MFCC 추출 루프
# ========================================

for idx, row in tqdm(df.iterrows(), total=len(df)):
    wav_path = row["wav_path"]

    try:
        # WAV 파일 로딩 (최대 max_duration 초까지만 로드)
        y, sr = librosa.load(wav_path, sr=sample_rate, duration=max_duration)

        # MFCC 추출 (13차원)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

        # 시간 축 기준으로 전치 (time_step, 13)
        mfcc_features.append(mfcc.T)
        labels.append(row["emotion"])

    except Exception as e:
        # 로딩 실패 시 에러 출력 및 로그 저장
        print(f"Error processing {wav_path}: {e}")
        error_files.append(wav_path)

    # 일정 개수 이상이면 배치 저장
    if len(mfcc_features) >= save_interval:
        np.save(os.path.join(save_dir, f"mfcc_batch_{save_counter}.npy"), np.array(mfcc_features, dtype=object))
        np.save(os.path.join(save_dir, f"label_batch_{save_counter}.npy"), np.array(labels))
        save_counter += 1
        mfcc_features = []
        labels = []

# 루프 종료 후 남은 데이터 저장
if mfcc_features:
    np.save(os.path.join(save_dir, f"mfcc_batch_{save_counter}.npy"), np.array(mfcc_features, dtype=object))
    np.save(os.path.join(save_dir, f"label_batch_{save_counter}.npy"), np.array(labels))

# ========================================
# 4. 에러 파일 저장
# ========================================

error_log_path = "/media/usou/PortableSSD/mldl_project/data/validation/broken_audio_files_val.txt"
with open(error_log_path, "w") as f:
    for path in error_files:
        f.write(path + "\n")

# ========================================
# 5. 처리 결과 출력
# ========================================

print(f"성공적으로 저장된 배치 수: {save_counter + 1}")
print(f"실패한 파일 수: {len(error_files)}")


# Validation 데이터용 레이블 인코딩

In [ ]:
import os
import numpy as np
import glob
import pickle
from sklearn.preprocessing import LabelEncoder

# ============================
# 1. 설정
# ============================
# 레이블 배치가 저장된 경로
label_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches"

# 인코딩된 레이블 저장 경로
encoded_label_dir = os.path.join(label_dir, "encoded_labels")
os.makedirs(encoded_label_dir, exist_ok=True)

# ============================
# 2. 모든 배치 레이블 수집
# ============================
# label_batch_*.npy 파일 경로 리스트
label_files = sorted(glob.glob(os.path.join(label_dir, "label_batch_*.npy")))

# 전체 레이블 리스트 생성
all_labels = []
batch_label_data = []  # 배치별 데이터도 임시 저장
for label_file in label_files:
    labels = np.load(label_file, allow_pickle=True)
    batch_label_data.append(labels)
    all_labels.extend(labels)

# ============================
# 3. 레이블 인코딩
# ============================
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

# 인코더 저장 (추후 예측 결과 복원용)
with open(os.path.join(label_dir, "label_encoder.pkl"), "wb") as f:
    pickle.dump(label_encoder, f)

# ============================
# 4. 인코딩된 레이블 배치별로 저장
# ============================
for i, labels in enumerate(batch_label_data):
    encoded = label_encoder.transform(labels)
    save_path = os.path.join(encoded_label_dir, f"label_batch_{i}.npy")
    np.save(save_path, encoded)

print(f"총 레이블 개수: {len(all_labels)}")
print(f"인코딩된 클래스 목록: {label_encoder.classes_}")
print(f"배치 수: {len(label_files)}")
print("레이블 인코딩 및 저장 완료")


# MFCC DataGenerator 클래스 생성

In [ ]:
import os
import numpy as np
import tensorflow as tf

class MFCCDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, batch_dir, prefix, batch_size=1, shuffle=True):
        self.batch_dir = batch_dir
        self.prefix = prefix
        self.batch_size = batch_size
        self.shuffle = shuffle

        # 배치 파일 목록 생성
        self.mfcc_files = sorted([
            f for f in os.listdir(batch_dir) if f.startswith(f"{prefix}_batch_")
        ])
        self.indices = list(range(len(self.mfcc_files)))
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_mfccs = []
        batch_labels = []

        for idx in batch_indices:
            mfcc_path = os.path.join(self.batch_dir, f"{self.prefix}_batch_{idx}.npy")
            label_path = os.path.join(self.batch_dir, "encoded_labels", f"label_batch_{idx}.npy")

            mfcc_data = np.load(mfcc_path, allow_pickle=True)
            label_data = np.load(label_path)

            # 가장 긴 시퀀스 기준으로 padding
            max_len = max([x.shape[0] for x in mfcc_data])
            padded = tf.keras.preprocessing.sequence.pad_sequences(mfcc_data, maxlen=max_len, dtype='float32', padding='post')
            padded = np.expand_dims(padded, -1)  # (batch, time, n_mfcc, 1)

            batch_mfccs.append(padded)
            batch_labels.append(label_data)

        X = np.concatenate(batch_mfccs, axis=0)
        y = np.concatenate(batch_labels, axis=0)
        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)


# GPU 활성화 및 안정 설정 코드

In [ ]:
import tensorflow as tf

# 1. GPU 장치 목록 출력
gpus = tf.config.list_physical_devices('GPU')
print("사용 가능한 GPU:", gpus)

# 2. 메모리 자동 증가 설정 (안정성을 위해 권장)
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✔ GPU 메모리 자동 증가 설정 완료")
    except RuntimeError as e:
        print("⚠ 메모리 설정 중 오류 발생:", e)
else:
    print("❌ GPU를 찾을 수 없습니다. CPU로 진행됩니다.")


# label_encoder.pkl을 로드해 동일하게 인코딩하는 코드

In [ ]:
import os
import numpy as np
import glob
import pickle

# ============================
# 1. 설정
# ============================
val_label_dir = "/media/usou/PortableSSD/mldl_project/data/validation/mfcc_batches"
encoded_label_dir = os.path.join(val_label_dir, "encoded_labels")
os.makedirs(encoded_label_dir, exist_ok=True)

# 학습 데이터에서 저장한 LabelEncoder 로드
with open("/media/usou/PortableSSD/mldl_project/data/mfcc_batches/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

# ============================
# 2. 모든 validation 레이블 수집
# ============================
label_files = sorted(glob.glob(os.path.join(val_label_dir, "label_batch_*.npy")))

for i, label_file in enumerate(label_files):
    labels = np.load(label_file, allow_pickle=True)
    encoded = label_encoder.transform(labels)
    save_path = os.path.join(encoded_label_dir, f"label_batch_{i}.npy")
    np.save(save_path, encoded)

print(f"✅ Validation 레이블 인코딩 및 저장 완료 (배치 수: {len(label_files)})")


# CNN 모델 정의

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_cnn_model(input_shape, num_classes):
    """
    CNN 기반 음성 감정 분류 모델 정의

    Parameters:
        input_shape (tuple): 입력 데이터 형태 (예: (시간축 길이, MFCC 차원 수, 채널 수))
        num_classes (int): 분류할 감정 클래스 수

    Returns:
        tensorflow.keras.Model: 컴파일 완료된 CNN 모델
    """
    model = models.Sequential()

    # [1] 첫 번째 컨볼루션 블록
    model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.BatchNormalization())               # 학습 안정성 향상
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))     # 공간 크기 감소

    # [2] 두 번째 컨볼루션 블록
    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # [3] 세 번째 컨볼루션 블록
    model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.GlobalAveragePooling2D())           # 피처맵 전체 평균값

    # [4] 완전 연결층 + 드롭아웃
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))                       # 과적합 방지

    # [5] 출력층 - 클래스 수만큼 softmax 출력
    model.add(layers.Dense(num_classes, activation='softmax'))

    # 모델 컴파일
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model


# 학습

In [ ]:
import tensorflow as tf

# ============================
# 1. 데이터 제너레이터 생성
# ============================
train_generator = MFCCDataGenerator(
    batch_dir="/media/usou/PortableSSD/mldl_project/data/mfcc_batches",
    prefix="mfcc",
    batch_size=1  # 메모리 안정 위해 소량으로 시작
)

val_generator = MFCCDataGenerator(
    batch_dir="/media/usou/PortableSSD/mldl_project/data/validation/mfcc_batches",
    prefix="mfcc",
    batch_size=1
)

# ============================
# 2. 클래스 수 설정
# ============================
import pickle

# 학습 데이터의 레이블 인코더를 불러와 클래스 수 확인
with open("/media/usou/PortableSSD/mldl_project/data/mfcc_batches/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

num_classes = len(label_encoder.classes_)

# ============================
# 3. 입력 형태 설정
# ============================
# 예시 입력 크기 지정 (임의 값, 실제 학습 데이터 확인 후 조정 가능)
# 이 부분은 train_generator[0][0].shape 로 확인 가능
sample_input = train_generator[0][0]  # shape: (batch, time, n_mfcc, 1)
input_shape = sample_input.shape[1:]  # (time, n_mfcc, 1)

# ============================
# 4. 모델 생성 및 요약
# ============================
model = create_cnn_model(input_shape=input_shape, num_classes=num_classes)
model.summary()

# ============================
# 5. 콜백 정의 (모델 저장 및 EarlyStopping)
# ============================
checkpoint_path = "/media/usou/PortableSSD/mldl_project/models/best_model.h5"
os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True
    )
]

# ============================
# 6. 학습 실행
# ============================
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=callbacks
)


- 학습 실패한 이유 
- Conv2D 커널이 너무 커서 작은 입력에 비해 작동을 못함 - kernel size 축소 or padding ="same" 적용 -> 모델 재정의

# 안정적인 CNN 모델 정의

In [ ]:
from tensorflow.keras import layers, models

def create_cnn_model(input_shape, num_classes):
    """
    음성 감정 분류를 위한 CNN 모델 정의
    
    Parameters:
        input_shape (tuple): 입력 데이터의 형태 (시간축, MFCC 차원, 채널 수)
        num_classes (int): 분류할 감정 클래스 수
        
    Returns:
        keras.models.Sequential: 컴파일된 모델 객체
    """

    model = models.Sequential()

    # ===============================
    # [1] 첫 번째 컨볼루션 블록
    # ===============================
    # Conv2D: 32개의 필터, 3x3 커널, relu 활성화 함수 사용
    # padding='same'으로 출력 크기 감소 방지
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.BatchNormalization())  # 정규화로 학습 안정화
    model.add(layers.MaxPooling2D((2, 2)))  # 출력 크기 절반으로 축소

    # ===============================
    # [2] 두 번째 컨볼루션 블록
    # ===============================
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # ===============================
    # [3] 세 번째 컨볼루션 블록
    # ===============================
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    
    # GlobalAveragePooling2D: 각 채널의 평균을 취해 1D 벡터로 변환
    model.add(layers.GlobalAveragePooling2D())

    # ===============================
    # [4] 완전 연결층 + 출력층
    # ===============================
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))  # 과적합 방지
    model.add(layers.Dense(num_classes, activation='softmax'))  # 감정 클래스 확률 출력

    # ===============================
    # [5] 모델 컴파일
    # ===============================
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


# 학습
- 학습 도중 시스템이 멈추지 않도록 작은 배치 크기와 적절한 콜백 설정 포함

In [ ]:
import os
import pickle
import tensorflow as tf

# ============================
# 1. 데이터 제너레이터 생성
# ============================
train_generator = MFCCDataGenerator(
    batch_dir="/media/usou/PortableSSD/mldl_project/data/mfcc_batches",
    prefix="mfcc",
    batch_size=1  # 메모리 절약을 위한 작은 배치
)

val_generator = MFCCDataGenerator(
    batch_dir="/media/usou/PortableSSD/mldl_project/data/validation/mfcc_batches",
    prefix="mfcc",
    batch_size=1
)

# ============================
# 2. 레이블 인코더 로드 및 클래스 수 설정
# ============================
# 학습 데이터용 레이블 인코더를 통해 클래스 수 파악
with open("/media/usou/PortableSSD/mldl_project/data/mfcc_batches/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

num_classes = len(label_encoder.classes_)

# ============================
# 3. 입력 형태 설정
# ============================
# 첫 배치에서 입력 형태 파악
sample_input = train_generator[0][0]  # shape: (batch, time, n_mfcc, 1)
input_shape = sample_input.shape[1:]  # (time, n_mfcc, 1)

# ============================
# 4. 모델 생성
# ============================
model = create_cnn_model(input_shape=input_shape, num_classes=num_classes)
model.summary()

# ============================
# 5. 콜백 설정
# ============================
checkpoint_path = "/media/usou/PortableSSD/mldl_project/models/best_model.h5"
os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True
    )
]

# ============================
# 6. 학습 실행
# ============================
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=callbacks
)


- GPU 메모리 부족

# 모델 경량화 프루닝 라이브러리 및 모델 정의

In [ ]:
import tensorflow_model_optimization as tfmot
from tensorflow.keras import layers, models

def create_pruned_cnn_model(input_shape, num_classes):
    # 프루닝 설정: 가중치의 50%를 0으로 만듦 (비율은 조절 가능)
    pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
            initial_sparsity=0.0,
            final_sparsity=0.5,
            begin_step=0,
            end_step=1000  # 조절 가능
        )
    }

    model = models.Sequential()

    # 첫 번째 블록
    model.add(tfmot.sparsity.keras.prune_low_magnitude(
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        **pruning_params
    ))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # 두 번째 블록
    model.add(tfmot.sparsity.keras.prune_low_magnitude(
        layers.Conv2D(64, (3, 3), activation='relu'),
        **pruning_params
    ))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # 세 번째 블록
    model.add(tfmot.sparsity.keras.prune_low_magnitude(
        layers.Conv2D(128, (3, 3), activation='relu'),
        **pruning_params
    ))
    model.add(layers.BatchNormalization())
    model.add(layers.GlobalAveragePooling2D())

    # 밀집층
    model.add(tfmot.sparsity.keras.prune_low_magnitude(
        layers.Dense(128, activation='relu'),
        **pruning_params
    ))
    model.add(layers.Dropout(0.3))
    model.add(tfmot.sparsity.keras.prune_low_magnitude(
        layers.Dense(num_classes, activation='softmax'),
        **pruning_params
    ))

    # 컴파일
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [ ]:
import tensorflow_model_optimization as tfmot
from tensorflow.keras import layers, models

def create_pruned_cnn_model(input_shape, num_classes):
    # 프루닝 설정: 가중치의 50%를 0으로 만듦 (비율은 조절 가능)
    pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
            initial_sparsity=0.0,
            final_sparsity=0.5,
            begin_step=0,
            end_step=1000  # 조절 가능
        )
    }

    model = models.Sequential()

    # 첫 번째 블록
    model.add(tfmot.sparsity.keras.prune_low_magnitude(
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        **pruning_params
    ))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # 두 번째 블록
    model.add(tfmot.sparsity.keras.prune_low_magnitude(
        layers.Conv2D(64, (3, 3), activation='relu'),
        **pruning_params
    ))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # 세 번째 블록
    model.add(tfmot.sparsity.keras.prune_low_magnitude(
        layers.Conv2D(128, (3, 3), activation='relu'),
        **pruning_params
    ))
    model.add(layers.BatchNormalization())
    model.add(layers.GlobalAveragePooling2D())

    # 밀집층
    model.add(tfmot.sparsity.keras.prune_low_magnitude(
        layers.Dense(128, activation='relu'),
        **pruning_params
    ))
    model.add(layers.Dropout(0.3))
    model.add(tfmot.sparsity.keras.prune_low_magnitude(
        layers.Dense(num_classes, activation='softmax'),
        **pruning_params
    ))

    # 컴파일
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model


- 프루닝 실패
1. Sequential 모델 안에 잘못된 레이어 구조를 넣었거나

2. 프루닝 대상에 이미 프루닝된 레이어를 다시 적용하려고 하거나

3. 모델 구조에서 무한 루프가 생겼거나

4. 너무 많은 프루닝 wrapper가 중첩된 경우



# 여기서 부터 

# MFCCDataGenerator 클래스
- tf.keras.utils.Sequence를 상속받아, 저장된 MFCC 및 레이블 배치 데이터를 Keras 모델 학습에 적합하게 동적으로 불러오고 전처리해주는 제너레이터

In [ ]:
import os
import numpy as np
import tensorflow as tf

class MFCCDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, batch_dir, prefix, batch_size=1, shuffle=True):
        self.batch_dir = batch_dir
        self.prefix = prefix
        self.batch_size = batch_size
        self.shuffle = shuffle

        # 배치 리스트 구성
        self.mfcc_files = sorted([
            f for f in os.listdir(batch_dir) if f.startswith(f"{prefix}_batch_")
        ])
        self.indices = list(range(len(self.mfcc_files)))
        if shuffle:
            np.random.shuffle(self.indices)

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, index):
        # 현재 배치 인덱스 범위 계산
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        batch_mfccs = []
        batch_labels = []

        for idx in batch_indices:
            mfcc_path = os.path.join(self.batch_dir, f"{self.prefix}_batch_{idx}.npy")
            label_path = os.path.join(self.batch_dir, "encoded_labels", f"label_batch_{idx}.npy")

            mfcc_data = np.load(mfcc_path, allow_pickle=True)
            label_data = np.load(label_path)

            # 시퀀스 길이 맞추기 (Zero-padding)
            max_len = max([x.shape[0] for x in mfcc_data])
            padded = tf.keras.preprocessing.sequence.pad_sequences(mfcc_data, maxlen=max_len, dtype='float32', padding='post')
            padded = np.expand_dims(padded, -1)  # CNN 입력 형식 맞추기

            batch_mfccs.append(padded)
            batch_labels.append(label_data)

        X = np.concatenate(batch_mfccs, axis=0)
        y = np.concatenate(batch_labels, axis=0)
        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)


# 음성 감정 분류를 위한 CNN 모델을 정의한 함수

In [ ]:
from tensorflow.keras import layers, models

def create_cnn_model(input_shape, num_classes):
    """
    음성 감정 인식을 위한 CNN 모델 정의
    - 입력: MFCC 시퀀스 (batch, time, n_mfcc, 1)
    - 출력: 감정 클래스 확률 (softmax)
    """

    model = models.Sequential()

    # [1] 첫 번째 컨볼루션 블록
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # [2] 두 번째 컨볼루션 블록
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # [3] 세 번째 컨볼루션 블록
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.GlobalAveragePooling2D())  # 피처맵 전체 평균값

    # [4] 완전 연결층
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(num_classes, activation='softmax'))  # 감정 클래스 개수만큼 출력

    # 모델 컴파일
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


# 학습

In [ ]:
import os
import pickle
import tensorflow as tf

# ===============================
# [0] GPU 메모리 설정 (OOM 방지)
# ===============================
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU memory growth enabled")
    except RuntimeError as e:
        print("❌ RuntimeError:", e)

# ===============================
# [1] 데이터 제너레이터 생성
# ===============================
train_generator = MFCCDataGenerator(
    batch_dir="/media/usou/PortableSSD/mldl_project/data/mfcc_batches",
    prefix="mfcc",
    batch_size=1  # 메모리 안전 위해 최소 배치
)

val_generator = MFCCDataGenerator(
    batch_dir="/media/usou/PortableSSD/mldl_project/data/validation/mfcc_batches",
    prefix="mfcc",
    batch_size=1
)

# ===============================
# [2] 레이블 인코더 로드 및 클래스 수
# ===============================
with open("/media/usou/PortableSSD/mldl_project/data/mfcc_batches/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

num_classes = len(label_encoder.classes_)

# ===============================
# [3] 입력 형태 확인
# ===============================
sample_input = train_generator[0][0]  # shape: (batch, time, n_mfcc, 1)
input_shape = sample_input.shape[1:]

# ===============================
# [4] 모델 생성
# ===============================
model = create_cnn_model(input_shape=input_shape, num_classes=num_classes)
model.summary()

# ===============================
# [5] 콜백 설정 (모델 저장 + 조기 종료)
# ===============================
checkpoint_path = "/media/usou/PortableSSD/mldl_project/models/best_model.h5"
os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True
    )
]

# ===============================
# [6] 모델 학습
# ===============================
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=callbacks
)


# 모델 변경  PyTorch 기반 간단한 CNN 모델 예시

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AudioEmotionCNN(nn.Module):
    def __init__(self, num_classes):
        super(AudioEmotionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


 # 데이터 로더 정의 (PyTorch용)

In [ ]:
from torch.utils.data import Dataset
import torch
import numpy as np
import os

class MFCCDataset(Dataset):
    def __init__(self, batch_dir, prefix):
        self.batch_dir = batch_dir
        self.prefix = prefix

        self.mfcc_files = sorted([
            f for f in os.listdir(batch_dir) if f.startswith(f"{prefix}_batch_")
        ])

    def __len__(self):
        return len(self.mfcc_files)

    def __getitem__(self, idx):
        mfcc_path = os.path.join(self.batch_dir, f"{self.prefix}_batch_{idx}.npy")
        label_path = os.path.join(self.batch_dir, "encoded_labels", f"label_batch_{idx}.npy")

        # 여기서도 배치 데이터임
        mfcc_batch = np.load(mfcc_path, allow_pickle=True)
        label_batch = np.load(label_path)

        # 리스트로 묶어서 반환 (collate_fn에서 처리)
        return list(zip(mfcc_batch, label_batch))


# collate_fn 추가 (패딩과 텐서 변환)

In [ ]:
def collate_fn(batch):
    batch = sum(batch, [])  # [(mfcc1, label1), ..., (mfccN, labelN)]로 평탄화
    seq_lens = [x[0].shape[0] for x in batch]
    max_len = max(seq_lens)
    n_mfcc = batch[0][0].shape[1]

    padded_mfccs = []
    labels = []

    for mfcc, label in batch:
        padded = np.zeros((max_len, n_mfcc), dtype=np.float32)
        padded[:mfcc.shape[0], :] = mfcc
        padded_mfccs.append(padded)
        labels.append(label)

    X = torch.tensor(padded_mfccs).unsqueeze(1)  # (batch, 1, time, n_mfcc)
    y = torch.tensor(labels, dtype=torch.long)
    return X, y


# DataLoader 생성

In [ ]:
train_dataset = MFCCDataset("/media/usou/PortableSSD/mldl_project/data/mfcc_batches", "mfcc")
val_dataset = MFCCDataset("/media/usou/PortableSSD/mldl_project/data/validation/mfcc_batches", "mfcc")

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)


# 모델 학습

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import os

# ============================
# 0. 기본 설정
# ============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

num_classes = len(label_encoder.classes_)
model = AudioEmotionCNN(num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 30

# ============================
# 1. 학습 루프
# ============================
best_val_acc = 0.0
save_path = "/media/usou/PortableSSD/mldl_project/models/best_model_pt.pth"
os.makedirs(os.path.dirname(save_path), exist_ok=True)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

    train_acc = correct / total
    print(f"🟢 Epoch {epoch+1}: Train Loss: {running_loss:.4f} | Train Acc: {train_acc:.4f}")

    # ============================
    # 2. 검증 루프
    # ============================
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, targets in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]"):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == targets).sum().item()
            val_total += targets.size(0)

    val_acc = val_correct / val_total
    print(f"🔵 Epoch {epoch+1}: Val Acc: {val_acc:.4f}")

    # ============================
    # 3. 모델 저장
    # ============================
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), save_path)
        print(f"💾 Best model saved with Val Acc: {val_acc:.4f}")

print("✅ 학습 완료")


- 에러 :  배치 안의 샘플들이 시퀀스 길이(time step)가 서로 달라서 torch.stack() 실패.
PyTorch DataLoader는 collate_fn이 내부에서 torch.stack()을 사용하기 때문에, 입력 데이터들 크기가 다르면 에러가 납니다.
TensorFlow에서는 padding으로 해결됐던 부분

- 해결 방안 : 배치 크기를 맞추는 collate_fn 함수 구현
    - collate_fn을 사용하여 배치 내 데이터 크기를 맞추는 방법을 추가

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import pickle

# ============================
# 0. 기본 설정
# ============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# ============================
# 1. 레이블 인코더 로드 및 클래스 수 확인
# ============================
with open("/media/usou/PortableSSD/mldl_project/data/mfcc_batches/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

num_classes = len(label_encoder.classes_)

# ============================
# 2. 데이터셋 클래스 정의
# ============================
class MFCCDataset(Dataset):
    def __init__(self, batch_dir, prefix):
        self.batch_dir = batch_dir
        self.prefix = prefix
        self.mfcc_files = sorted([f for f in os.listdir(batch_dir) if f.startswith(f"{prefix}_batch_")])

    def __len__(self):
        return len(self.mfcc_files)

    def __getitem__(self, idx):
        mfcc_path = os.path.join(self.batch_dir, f"{self.prefix}_batch_{idx}.npy")
        label_path = os.path.join(self.batch_dir, "encoded_labels", f"label_batch_{idx}.npy")

        mfcc_data = np.load(mfcc_path, allow_pickle=True)
        label_data = np.load(label_path)

        # 리스트 형태일 경우 numpy 배열로 변환
        if isinstance(mfcc_data, list):
            mfcc_data = np.array(mfcc_data)

        # 2D 배열인지 확인 (time, n_mfcc)
        if mfcc_data.ndim != 2:
            raise ValueError(f"[{mfcc_path}] shape 오류: (time, n_mfcc) 형식이 아님 → 실제 shape: {mfcc_data.shape}")

        return mfcc_data, label_data


# 패딩을 위한 collate_fn 정의
def collate_fn(batch):
    # 각 샘플은 (sequence_len, n_mfcc)
    seq_lens = [sample[0].shape[0] for sample in batch]
    max_len = max(seq_lens)

    padded_batch = []
    labels = []

    for mfcc_data, label_data in batch:
        # mfcc_data shape: (time, n_mfcc)
        time_len = mfcc_data.shape[0]
        n_mfcc = mfcc_data.shape[1]

        # (time, n_mfcc) → (max_len, n_mfcc)
        padded = np.zeros((max_len, n_mfcc), dtype=np.float32)
        padded[:time_len, :] = mfcc_data

        padded_batch.append(padded)
        labels.append(label_data)

    # (batch, 1, time, n_mfcc)
    X = torch.tensor(padded_batch).unsqueeze(1)
    y = torch.tensor(labels, dtype=torch.long)
    return X, y


# 데이터셋 로딩
train_dataset = MFCCDataset(batch_dir="/media/usou/PortableSSD/mldl_project/data/mfcc_batches", prefix="mfcc")
val_dataset = MFCCDataset(batch_dir="/media/usou/PortableSSD/mldl_project/data/validation/mfcc_batches", prefix="mfcc")

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

# ============================
# 3. 모델 정의
# ============================
class AudioEmotionCNN(nn.Module):
    def __init__(self, num_classes):
        super(AudioEmotionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# 모델과 손실 함수, 옵티마이저 설정
model = AudioEmotionCNN(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 30

# ============================
# 4. 학습 루프
# ============================
best_val_acc = 0.0
save_path = "/media/usou/PortableSSD/mldl_project/models/best_model_pt.pth"
os.makedirs(os.path.dirname(save_path), exist_ok=True)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

    train_acc = correct / total
    print(f"🟢 Epoch {epoch+1}: Train Loss: {running_loss:.4f} | Train Acc: {train_acc:.4f}")

    # ============================
    # 5. 검증 루프
    # ============================
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, targets in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]"):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == targets).sum().item()
            val_total += targets.size(0)

    val_acc = val_correct / val_total
    print(f"🔵 Epoch {epoch+1}: Val Acc: {val_acc:.4f}")

    # ============================
    # 6. 모델 저장
    # ============================
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), save_path)
        print(f"💾 Best model saved with Val Acc: {val_acc:.4f}")

print("✅ 학습 완료")


- 에러
allow_pickle=True 옵션으로 불러온 데이터를 np.load() 하면, 원래는 List[np.ndarray] 혹은 (time, n_mfcc) 구조여야 합니다.

만약 이전에 이 mfcc_batch_1.npy를 배치 단위 리스트 형태로 저장했었다면:

지금처럼 __getitem__에서 개별 .npy를 꺼낼 경우 배치 전체가 한 개의 1D 배열로 저장되어 있을 수 있습니다.

- 해결 방법: 각 샘플을 개별 .npy 파일로 저장

In [ ]:
import numpy as np
import os

# 원본 다중샘플 npy 경로
input_path = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches/mfcc_batch_1.npy"

# 저장할 경로
output_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches/split_samples"
os.makedirs(output_dir, exist_ok=True)

# 데이터 로드
data = np.load(input_path, allow_pickle=True)

# 각 샘플 저장
for i, sample in enumerate(data):
    save_path = os.path.join(output_dir, f"sample_{i:04d}.npy")
    np.save(save_path, sample)

print(f"✅ 총 {len(data)}개 샘플 저장 완료: {output_dir}")


# 레이블 분할 저장 코드

In [ ]:
import numpy as np
import os

# 레이블 경로
label_path = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches/encoded_labels/label_batch_1.npy"
labels = np.load(label_path)

# 저장할 디렉토리
label_output_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches/split_labels"
os.makedirs(label_output_dir, exist_ok=True)

# 분할 저장
for i, label in enumerate(labels):
    save_path = os.path.join(label_output_dir, f"label_{i:04d}.npy")
    np.save(save_path, label)

print(f"✅ 총 {len(labels)}개 레이블 저장 완료: {label_output_dir}")


# MFCCSampleDataset 정의 (샘플 단위)

In [ ]:
from torch.utils.data import Dataset
import numpy as np
import os
import torch

class MFCCSampleDataset(Dataset):
    def __init__(self, sample_dir, label_dir):
        self.sample_paths = sorted([
            os.path.join(sample_dir, f) for f in os.listdir(sample_dir) if f.endswith(".npy")
        ])
        self.label_paths = sorted([
            os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith(".npy")
        ])

    def __len__(self):
        return len(self.sample_paths)

    def __getitem__(self, idx):
        mfcc = np.load(self.sample_paths[idx])          # shape: (time, n_mfcc)
        label = np.load(self.label_paths[idx])          # shape: ()

        # 텐서로 변환 (채널 추가)
        mfcc_tensor = torch.tensor(mfcc, dtype=torch.float32).unsqueeze(0)  # (1, time, n_mfcc)
        label_tensor = torch.tensor(label, dtype=torch.long)

        return mfcc_tensor, label_tensor


# collate_fn 정의

In [ ]:
def collate_fn(batch):
    seq_lens = [x[0].shape[0] for x in batch]
    max_len = max(seq_lens)

    # n_mfcc 추정 시 더 안전하게
    n_mfcc = None
    for mfcc_data, _ in batch:
        if isinstance(mfcc_data, np.ndarray) and mfcc_data.ndim == 2:
            n_mfcc = mfcc_data.shape[1]
            break

    if n_mfcc is None:
        raise ValueError("모든 샘플에서 유효한 2D MFCC 데이터를 찾을 수 없습니다.")

    padded_batch = []
    labels = []

    for mfcc_data, label_data in batch:
        time_len = mfcc_data.shape[0]
        padded = np.zeros((max_len, n_mfcc), dtype=np.float32)
        padded[:time_len, :] = mfcc_data
        padded_batch.append(padded)
        labels.append(label_data)

    X = torch.tensor(padded_batch).unsqueeze(1)  # (batch, 1, time, n_mfcc)
    y = torch.tensor(labels, dtype=torch.long)
    return X, y


# DataLoader 생성

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=collate_fn
)

val_loader = DataLoader(
    val_dataset,
    batch_size=4,
    shuffle=False,
    collate_fn=collate_fn
)


#  기본 설정 및 모델 불러오기

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# 모델 정의 (이전에 정의한 AudioEmotionCNN 사용)
model = AudioEmotionCNN(num_classes=8).to(device)  # 클래스 수에 맞게 수정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 30


# 학습 루프

In [ ]:
best_val_acc = 0.0
save_path = "/media/usou/PortableSSD/mldl_project/models/best_model_pt.pth"
os.makedirs(os.path.dirname(save_path), exist_ok=True)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

    train_acc = correct / total
    print(f"🟢 Epoch {epoch+1}: Train Loss: {running_loss:.4f} | Train Acc: {train_acc:.4f}")


# Tensorflow 로 변경

# 1. 샘플 단위 .npy 파일을 위한 DataGenerator 정의

In [ ]:
import os
import numpy as np
import tensorflow as tf

class SampleMFCCDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, mfcc_dir, label_dir, batch_size=32, shuffle=True):
        self.mfcc_paths = sorted([
            os.path.join(mfcc_dir, f) for f in os.listdir(mfcc_dir) if f.endswith(".npy")
        ])
        self.label_paths = sorted([
            os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith(".npy")
        ])
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.mfcc_paths))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.mfcc_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = [], []

        for i in batch_indices:
            mfcc = np.load(self.mfcc_paths[i])  # (time, n_mfcc)
            label = np.load(self.label_paths[i])  # 정수 인코딩 레이블

            X.append(mfcc)
            y.append(label)

        # Zero-padding
        max_len = max(x.shape[0] for x in X)
        X_pad = tf.keras.preprocessing.sequence.pad_sequences(
            X, maxlen=max_len, padding='post', dtype='float32'
        )
        X_pad = np.expand_dims(X_pad, -1)  # (batch, time, n_mfcc, 1)
        y = np.array(y)

        return X_pad, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)


# CNN 모델 정의

In [ ]:
from tensorflow.keras import layers, models

def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.GlobalAveragePooling2D())

    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


# 2.1  validation 데이터도 split 

In [ ]:
import numpy as np
import os

val_input_path = "/media/usou/PortableSSD/mldl_project/data/validation/mfcc_batches/mfcc_batch_0.npy"
val_label_path = "/media/usou/PortableSSD/mldl_project/data/validation/mfcc_batches/label_batch_0.npy"

# 저장할 폴더
val_sample_dir = "/media/usou/PortableSSD/mldl_project/data/validation/mfcc_batches/split_samples"
val_label_dir = "/media/usou/PortableSSD/mldl_project/data/validation/mfcc_batches/split_labels"
os.makedirs(val_sample_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

# 데이터 로드
mfcc_data = np.load(val_input_path, allow_pickle=True)
label_data = np.load(val_label_path)

# 저장
for i, (sample, label) in enumerate(zip(mfcc_data, label_data)):
    np.save(os.path.join(val_sample_dir, f"sample_{i:04d}.npy"), sample)
    np.save(os.path.join(val_label_dir, f"label_{i:04d}.npy"), label)

print(f"✅ validation용 {len(mfcc_data)}개 샘플 및 레이블 저장 완료")


# 3. 콜백 설정 및 모델 학습

In [ ]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# ===============================
# [1] 학습 및 검증용 DataGenerator 정의
# ===============================
train_generator = SampleMFCCDataGenerator(
    mfcc_dir="/media/usou/PortableSSD/mldl_project/data/mfcc_batches/split_samples",
    label_dir="/media/usou/PortableSSD/mldl_project/data/mfcc_batches/split_labels",
    batch_size=8
)

val_generator = SampleMFCCDataGenerator(
    mfcc_dir="/media/usou/PortableSSD/mldl_project/data/validation/mfcc_batches/split_samples",
    label_dir="/media/usou/PortableSSD/mldl_project/data/validation/mfcc_batches/split_labels",
    batch_size=8
)

# ===============================
# [2] 입력 형태 확인 및 모델 생성
# ===============================
sample_input, _ = train_generator[0]
input_shape = sample_input.shape[1:]  # (time, n_mfcc, 1)

# 클래스 수 확인
import glob
label_paths = sorted(glob.glob("/media/usou/PortableSSD/mldl_project/data/mfcc_batches/split_labels/*.npy"))
all_labels = [int(np.load(p)) for p in label_paths]
num_classes = len(set(all_labels))

model = create_cnn_model(input_shape=input_shape, num_classes=num_classes)
model.summary()

# ===============================
# [3] 콜백 설정
# ===============================
checkpoint_path = "/media/usou/PortableSSD/mldl_project/models/best_model_tf.h5"
os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)

callbacks = [
    ModelCheckpoint(
        filepath=checkpoint_path,
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True
    )
]

# GPU 초기화를 위한 예열용 더미 실행 (CPU에서 실행)
with tf.device("/CPU:0"):
    model(tf.random.normal((1,) + input_shape))

# ===============================
# [4] 모델 학습 실행
# ===============================
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=callbacks
)

print("✅ 모델 학습 완료 및 저장 완료")


# gpu 사용 확인

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


# cpu로 학습 시도

In [ ]:
# 0. GPU 완전 비활성화 (가장 먼저 실행)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# 1. 필수 라이브러리 임포트
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import layers, models
import glob

# 2. 사용자 정의 DataGenerator
class SampleMFCCDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, mfcc_dir, label_dir, batch_size=32, shuffle=True):
        self.mfcc_paths = sorted([
            os.path.join(mfcc_dir, f) for f in os.listdir(mfcc_dir) if f.endswith(".npy")
        ])
        self.label_paths = sorted([
            os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith(".npy")
        ])
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.mfcc_paths))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.mfcc_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_mfcc = [np.load(self.mfcc_paths[i]) for i in batch_indexes]
        batch_label = [np.load(self.label_paths[i]).item() for i in batch_indexes]  # .item() 추가

        batch_mfcc = [np.expand_dims(x, axis=-1) for x in batch_mfcc]  # (time, n_mfcc, 1)

        max_len = max(x.shape[0] for x in batch_mfcc)
        padded_mfcc = np.array([
            np.pad(x, ((0, max_len - x.shape[0]), (0, 0), (0, 0)), mode='constant')
            for x in batch_mfcc
        ])

        labels = np.array(batch_label, dtype=np.int32)
        return padded_mfcc, labels

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

# 3. CNN 모델 생성 함수
def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.GlobalAveragePooling2D())

    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# 4. DataGenerator 설정
train_generator = SampleMFCCDataGenerator(
    mfcc_dir="/media/usou/PortableSSD/mldl_project/data/mfcc_batches/split_samples",
    label_dir="/media/usou/PortableSSD/mldl_project/data/mfcc_batches/split_labels",
    batch_size=8
)

val_generator = SampleMFCCDataGenerator(
    mfcc_dir="/media/usou/PortableSSD/mldl_project/data/validation/mfcc_batches/split_samples",
    label_dir="/media/usou/PortableSSD/mldl_project/data/validation/mfcc_batches/split_labels",
    batch_size=8
)

# 5. 입력 형상 및 클래스 수 확인
sample_input, _ = train_generator[0]
input_shape = sample_input.shape[1:]

label_paths = sorted(glob.glob("/media/usou/PortableSSD/mldl_project/data/mfcc_batches/split_labels/*.npy"))
all_labels = [np.load(p).item() for p in label_paths]  # .item()으로 스칼라 추출
num_classes = len(set(all_labels))
print(f"클래스 수: {num_classes}")

# 6. 모델 생성 및 콜백 설정
model = create_cnn_model(input_shape=input_shape, num_classes=num_classes)

checkpoint_path = "/media/usou/PortableSSD/mldl_project/models/best_model_tf.h5"
os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)

callbacks = [
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
]

# 7. 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=callbacks
)

print("모델 학습 완료 및 저장 완료")


# 머신 러닝 시도
## 작업 순서
1. 데이터 로딩 및 통합
    - metadata_cleaned (train)
    - metadata_cleaned_val (validation)

2. 특징(X)과 레이블(y) 분리
    - MFCC는 X
    - 감정 레이블은 y

3. 레이블 인코딩
    - 문자열 레이블(Happy, Sad 등)을 숫자로 변환

4. 머신러닝 모델 선택 및 학습
    - 예: RandomForestClassifier, SVC, GradientBoosting, LogisticRegression

5. 검증 및 평가
    - accuracy_score, classification_report, confusion_matrix

# 데이터 로딩 및 통합

In [ ]:
import numpy as np
import os
from glob import glob

# 경로
train_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches/metadata_cleaned"
val_dir = "/media/usou/PortableSSD/mldl_project/data/validation/mfcc_batches/metadata_cleaned_val"

# 파일 불러오기
train_mfcc_paths = sorted(glob(os.path.join(train_dir, "*.npy")))
val_mfcc_paths = sorted(glob(os.path.join(val_dir, "*.npy")))

# 데이터 로딩
train_data = [np.load(f, allow_pickle=True).item() for f in train_mfcc_paths]
val_data = [np.load(f, allow_pickle=True).item() for f in val_mfcc_paths]

# 분리
X_train = [d["mfcc"] for d in train_data]
y_train = [d["emotion"] for d in train_data]

X_val = [d["mfcc"] for d in val_data]
y_val = [d["emotion"] for d in val_data]


# 특징(X)과 레이블(y) 분리

In [ ]:
import pandas as pd

# 학습용 데이터 로딩
train_path = "/media/usou/PortableSSD/mldl_project/data/metadata_cleaned.csv"
metadata_cleaned = pd.read_csv(train_path)

# 검증용 데이터 로딩
val_path = "/media/usou/PortableSSD/mldl_project/data/validation/metadata_cleaned_val.csv"
metadata_cleaned_val = pd.read_csv(val_path)

# X: MFCC 특징, y: 감정 레이블
X_train = metadata_cleaned["mfcc"]
y_train = metadata_cleaned["emotion"]

X_val = metadata_cleaned_val["mfcc"]
y_val = metadata_cleaned_val["emotion"]


In [ ]:
print(metadata_cleaned.columns)


In [ ]:
base_mfcc_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches/split_samples"

def load_mfcc_from_path(path_series):
    mfcc_list = []
    for rel_path in path_series:
        fname = os.path.splitext(os.path.basename(rel_path))[0]
        mfcc_path = os.path.join(base_mfcc_dir, f"{fname}.npy")
        if not os.path.exists(mfcc_path):
            print(f"⚠️ 누락된 MFCC 파일: {mfcc_path}")
            continue
        mfcc = np.load(mfcc_path)
        mfcc_list.append(mfcc.flatten())  # 머신러닝용 1D 벡터로
    return np.array(mfcc_list)


In [ ]:
# X (특징) 로딩
X_train = load_mfcc_from_path(metadata_cleaned["wav_path"])
X_val = load_mfcc_from_path(metadata_cleaned_val["wav_path"])

# y (레이블) 추출
y_train = metadata_cleaned["emotion"].values
y_val = metadata_cleaned_val["emotion"].values


# MFCC 추출 이후 부터 딥러닝 다시 

## 데이터 로딩 및 전처리 

In [ ]:
import numpy as np
import os
from glob import glob

# 경로 설정
mfcc_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches"
label_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches/encoded_labels"

# 파일 경로 정렬
mfcc_paths = sorted(glob(os.path.join(mfcc_dir, "mfcc_batch_*.npy")))
label_paths = sorted(glob(os.path.join(label_dir, "label_batch_*.npy")))

# 전체 로딩
X_raw = [np.load(p, allow_pickle=True) for p in mfcc_paths]
y_raw = [np.load(p) for p in label_paths]

# 리스트로 되어 있는 MFCC들을 한 리스트로 합치기
X_all = [sample for batch in X_raw for sample in batch]
y_all = np.concatenate(y_raw)

print(f"총 MFCC 샘플 수: {len(X_all)}")
print(f"총 레이블 수: {len(y_all)}")


# MFCC 샘플 길이 정규화(패딩)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# MFCC의 각 샘플은 (time, 13) 형태 → 시퀀스 길이 맞추기
max_len = max([x.shape[0] for x in X_all])
print("가장 긴 MFCC 시퀀스 길이:", max_len)

# (샘플 수, time, n_mfcc)
X_padded = pad_sequences(X_all, maxlen=max_len, padding='post', dtype='float32')
print("패딩된 X shape:", X_padded.shape)

# 마지막 차원을 명시적으로 추가: (샘플 수, time, n_mfcc, 1)
X_padded = np.expand_dims(X_padded, -1)


In [ ]:
# 검증용 MFCC 데이터 로딩
val_mfcc_dir = "/media/usou/PortableSSD/mldl_project/data/validation/mfcc_batches"
val_label_dir = "/media/usou/PortableSSD/mldl_project/data/validation/mfcc_batches/encoded_labels"

val_mfcc_paths = sorted(glob(os.path.join(val_mfcc_dir, "mfcc_batch_*.npy")))
val_label_paths = sorted(glob(os.path.join(val_label_dir, "label_batch_*.npy")))

X_val_raw = [np.load(p, allow_pickle=True) for p in val_mfcc_paths]
y_val_raw = [np.load(p) for p in val_label_paths]

X_val_all = [sample for batch in X_val_raw for sample in batch]
y_val = np.concatenate(y_val_raw)

print(f"검증용 MFCC 샘플 수: {len(X_val_all)}")
print(f"검증용 레이블 수: {len(y_val)}")


# CNN 기반 모델 정의

In [ ]:
from tensorflow.keras import models, layers

def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.GlobalAveragePooling2D())

    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
input_shape = X_padded.shape[1:]  # (157, 13, 1)
num_classes = len(np.unique(y_all))

model = create_cnn_model(input_shape=input_shape, num_classes=num_classes)
model.summary()


# 콜백 설정 & 모델 학습 준비

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# 저장 경로
checkpoint_path = "/media/usou/PortableSSD/mldl_project/models/best_model_cnn.h5"
os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)

# 콜백 설정
callbacks = [
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
]


# 학습

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=64,
    epochs=30,
    callbacks=callbacks
)


# 커널 재시작 후

In [ ]:
# 1. MFCC 및 레이블 로딩
import numpy as np
import os
from glob import glob
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# 경로
mfcc_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches"
label_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches/encoded_labels"

# 데이터 로딩
mfcc_paths = sorted(glob(os.path.join(mfcc_dir, "mfcc_batch_*.npy")))
label_paths = sorted(glob(os.path.join(label_dir, "label_batch_*.npy")))
X_raw = [np.load(p, allow_pickle=True) for p in mfcc_paths]
y_raw = [np.load(p) for p in label_paths]

X_all = [sample for batch in X_raw for sample in batch]
y_all = np.concatenate(y_raw)

# 시퀀스 패딩
max_len = 157  # 고정
X_padded = pad_sequences(X_all, maxlen=max_len, padding='post', dtype='float32')
X_padded = np.expand_dims(X_padded, -1)

# 훈련/검증 분리
X_train, X_val, y_train, y_val = train_test_split(
    X_padded, y_all, test_size=0.12, random_state=42, stratify=y_all
)

print("✅ 데이터 준비 완료")


- 이 상황은 매우 자주 발생하는 Jupyter Notebook의 커널 메모리 한계 문제

- 위 방법으로 해결 예저
1. generator 정의
2. gc.collect()
3. model.fit(generator, validation_data=(X_val, y_val))



# 배치 제너레이터 정의(훈련용)

In [ ]:
import numpy as np
import tensorflow as tf  # ✅ 반드시 필요


In [ ]:
class MFCCBatchGenerator(tf.keras.utils.Sequence):
    def __init__(self, mfcc_paths, label_paths, batch_size=16, max_len=157, shuffle=True):
        self.mfcc_paths = mfcc_paths
        self.label_paths = label_paths
        self.batch_size = batch_size
        self.max_len = max_len
        self.shuffle = shuffle
        self.indices = np.arange(len(self.mfcc_paths))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.mfcc_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_idx = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        X_batch = []
        y_batch = []

        for i in batch_idx:
            mfcc = np.load(self.mfcc_paths[i], allow_pickle=True)
            label = np.load(self.label_paths[i])

            # 유효한 MFCC만 처리
            if isinstance(mfcc, np.ndarray) and len(mfcc.shape) == 2 and mfcc.shape[1] == 13:
                padded = tf.keras.preprocessing.sequence.pad_sequences(
                    [mfcc], maxlen=self.max_len, padding='post', dtype='float32'
                )[0]
                X_batch.append(padded)
                y_batch.append(label)

        X_batch = np.expand_dims(np.array(X_batch), -1)  # (batch, time, n_mfcc, 1)
        y_batch = np.array(y_batch)

        return X_batch, y_batch

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)


# 제너레이터 생성 및 모델 학습

In [ ]:
import gc
import tensorflow as tf

# 경로 설정
mfcc_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches"
label_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches/encoded_labels"

from glob import glob
import os

mfcc_paths = sorted(glob(os.path.join(mfcc_dir, "mfcc_batch_*.npy")))
label_paths = sorted(glob(os.path.join(label_dir, "label_batch_*.npy")))

# 학습용 제너레이터
train_generator = MFCCBatchGenerator(mfcc_paths, label_paths, batch_size=64, max_len=157, shuffle=True)

# 가비지 컬렉션 실행 (메모리 확보)
gc.collect()

from tensorflow.keras import models, layers

def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.GlobalAveragePooling2D())

    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model


# 모델 정의
input_shape = (157, 13, 1)
num_classes = 8
model = create_cnn_model(input_shape=input_shape, num_classes=num_classes)


# 콜백 정의
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint_path = "/media/usou/PortableSSD/mldl_project/models/best_model_generator.h5"
callbacks = [
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
]

# 검증용 데이터는 메모리 적재 방식으로 유지
# 이 부분은 이전에 분리한 X_val, y_val을 사용해야 합니다
# 혹시 없다면 validation set 따로 만들 수 있도록 알려주세요

In [ ]:
# 전체 레이블 로딩
all_labels = [np.load(p) for p in label_paths]
y_all = np.concatenate(all_labels)

# 전체 MFCC 로딩 (메모리 작은 검증셋만 로딩)
val_size = 112157  # 예: 전체의 약 13%
X_val_paths = mfcc_paths[-val_size:]
y_val_paths = label_paths[-val_size:]

# 검증 데이터 로딩
X_val_list = []
for path in X_val_paths:
    mfcc = np.load(path, allow_pickle=True)
    padded = tf.keras.preprocessing.sequence.pad_sequences(
        [mfcc], maxlen=157, padding='post', dtype='float32'
    )[0]
    X_val_list.append(padded)

X_val = np.expand_dims(np.array(X_val_list), -1)
y_val = np.concatenate([np.load(p) for p in y_val_paths])


In [ ]:
X_val_list = []
valid_y_list = []

for mfcc_path, label_path in zip(X_val_paths, y_val_paths):
    mfcc = np.load(mfcc_path, allow_pickle=True)
    
    # (time, 13) 형식인지 확인
    if isinstance(mfcc, np.ndarray) and len(mfcc.shape) == 2 and mfcc.shape[1] == 13:
        padded = tf.keras.preprocessing.sequence.pad_sequences(
            [mfcc], maxlen=157, padding='post', dtype='float32'
        )[0]
        X_val_list.append(padded)

        label = np.load(label_path)
        valid_y_list.append(label)
    else:
        print("❌ 잘못된 MFCC:", mfcc_path)

# 최종 배열로 변환
X_val = np.expand_dims(np.array(X_val_list), -1)
y_val = np.array(valid_y_list)

print("검증용 X shape:", X_val.shape)
print("검증용 y shape:", y_val.shape)


In [ ]:
import numpy as np
import tensorflow as tf
from glob import glob
import os

# 경로
mfcc_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches"
label_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches/encoded_labels"

# 배치 파일 리스트
mfcc_batch_paths = sorted(glob(os.path.join(mfcc_dir, "mfcc_batch_*.npy")))
label_batch_paths = sorted(glob(os.path.join(label_dir, "label_batch_*.npy")))

# 검증용 비율 기준으로 뒤에서 N개 배치만 선택
val_batch_count = 10  # 예: 마지막 10개 배치만 사용
X_val_list = []
y_val_list = []

for mfcc_batch_path, label_batch_path in zip(mfcc_batch_paths[-val_batch_count:], label_batch_paths[-val_batch_count:]):
    mfcc_batch = np.load(mfcc_batch_path, allow_pickle=True)
    label_batch = np.load(label_batch_path)

    for mfcc, label in zip(mfcc_batch, label_batch):
        if isinstance(mfcc, np.ndarray) and len(mfcc.shape) == 2 and mfcc.shape[1] == 13:
            padded = tf.keras.preprocessing.sequence.pad_sequences(
                [mfcc], maxlen=157, padding='post', dtype='float32'
            )[0]
            X_val_list.append(padded)
            y_val_list.append(label)

# 배열로 변환
X_val = np.expand_dims(np.array(X_val_list), -1)
y_val = np.array(y_val_list)

print("✅ 검증용 X shape:", X_val.shape)
print("✅ 검증용 y shape:", y_val.shape)


In [ ]:
history = model.fit(
    train_generator,
    validation_data=(X_val, y_val),  # 이 부분이 메모리에 있어야 해요!
    epochs=30,
    callbacks=callbacks
)


# 다시 정리 커널 재시작 후

# 1 

In [ ]:
import numpy as np
import tensorflow as tf
import os, gc
from glob import glob
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


# 2

In [ ]:
mfcc_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches"
label_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches/encoded_labels"

mfcc_paths = sorted(glob(os.path.join(mfcc_dir, "mfcc_batch_*.npy")))
label_paths = sorted(glob(os.path.join(label_dir, "label_batch_*.npy")))


# 3

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# 검증용 샘플 수 고정
val_sample_size = 112157

X_val_list = []
y_val_list = []
loaded = 0

for mfcc_path, label_path in zip(reversed(mfcc_paths), reversed(label_paths)):
    mfcc_batch = np.load(mfcc_path, allow_pickle=True)
    label_batch = np.load(label_path)

    for mfcc, label in zip(mfcc_batch, label_batch):
        if isinstance(mfcc, np.ndarray) and len(mfcc.shape) == 2 and mfcc.shape[1] == 13:
            padded = pad_sequences([mfcc], maxlen=157, padding='post', dtype='float32')[0]
            X_val_list.append(padded)
            y_val_list.append(label)
            loaded += 1
            if loaded >= val_sample_size:
                break
    if loaded >= val_sample_size:
        break

# numpy 배열로 변환 (꼭 확인!)
X_val = np.array(X_val_list).reshape(-1, 157, 13, 1)
y_val = np.array(y_val_list)

print("✅ 검증 데이터 재구성 완료")
print("검증용 X shape:", X_val.shape)
print("검증용 y shape:", y_val.shape)


# 4

In [ ]:
class MFCCBatchGenerator(tf.keras.utils.Sequence):
    def __init__(self, mfcc_paths, label_paths, batch_size=16, max_len=157, shuffle=True):
        self.mfcc_paths = mfcc_paths
        self.label_paths = label_paths
        self.max_len = max_len
        self.batch_size = batch_size
        self.shuffle = shuffle

        # 전체 샘플을 flatten
        self.X_all = []
        self.y_all = []

        for mfcc_path, label_path in zip(mfcc_paths, label_paths):
            mfcc_batch = np.load(mfcc_path, allow_pickle=True)
            label_batch = np.load(label_path)

            for mfcc, label in zip(mfcc_batch, label_batch):
                if isinstance(mfcc, np.ndarray) and mfcc.ndim == 2 and mfcc.shape[1] == 13:
                    self.X_all.append(mfcc)
                    self.y_all.append(label)

        self.indices = np.arange(len(self.X_all))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.X_all) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        X_batch = []
        y_batch = []

        for i in batch_indices:
            mfcc = self.X_all[i]
            padded = tf.keras.preprocessing.sequence.pad_sequences(
                [mfcc], maxlen=self.max_len, padding='post', dtype='float32'
            )[0]
            X_batch.append(padded)
            y_batch.append(self.y_all[i])

        X_batch = np.array(X_batch).reshape(-1, 157, 13, 1)
        y_batch = np.array(y_batch)
        return X_batch, y_batch

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)


# 5

In [ ]:
from tensorflow.keras import models, layers

def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.GlobalAveragePooling2D())

    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model


# 6

In [ ]:
input_shape = (157, 13, 1)
num_classes = 8
model = create_cnn_model(input_shape, num_classes)

checkpoint_path = "/media/usou/PortableSSD/mldl_project/models/best_model_generator.h5"
callbacks = [
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
]


# 7

In [ ]:
history = model.fit(
    train_generator,
    validation_data=(X_val, y_val),
    epochs=30,
    callbacks=callbacks
)


In [ ]:
# 1. 기본 라이브러리
import numpy as np
import tensorflow as tf
import os, gc
from glob import glob
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


# 2. 경로 설정
mfcc_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches"
label_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches/encoded_labels"

mfcc_paths = sorted(glob(os.path.join(mfcc_dir, "mfcc_batch_*.npy")))
label_paths = sorted(glob(os.path.join(label_dir, "label_batch_*.npy")))

# 3. 검증 데이터 구성
val_sample_size = 112157
X_val_list, y_val_list = [], []
loaded = 0

for mfcc_path, label_path in zip(reversed(mfcc_paths), reversed(label_paths)):
    mfcc_batch = np.load(mfcc_path, allow_pickle=True)
    label_batch = np.load(label_path)
    
    for mfcc, label in zip(mfcc_batch, label_batch):
        if isinstance(mfcc, np.ndarray) and mfcc.ndim == 2 and mfcc.shape[1] == 13:
            padded = pad_sequences([mfcc], maxlen=157, padding='post', dtype='float32')[0]
            X_val_list.append(padded)
            y_val_list.append(label)
            loaded += 1
            if loaded >= val_sample_size:
                break
    if loaded >= val_sample_size:
        break

X_val = np.array(X_val_list).reshape(-1, 157, 13, 1)
y_val = np.array(y_val_list)
print("✅ 검증 데이터 준비 완료")
print("검증용 X shape:", X_val.shape)
print("검증용 y shape:", y_val.shape)

# 4. 학습용 제너레이터 정의 및 생성
class MFCCBatchGenerator(tf.keras.utils.Sequence):
    def __init__(self, mfcc_paths, label_paths, batch_size=16, max_len=157, shuffle=True):
        self.max_len = max_len
        self.batch_size = batch_size
        self.shuffle = shuffle

        self.X_all = []
        self.y_all = []

        for mfcc_path, label_path in zip(mfcc_paths, label_paths):
            mfcc_batch = np.load(mfcc_path, allow_pickle=True)
            label_batch = np.load(label_path)
            for mfcc, label in zip(mfcc_batch, label_batch):
                if isinstance(mfcc, np.ndarray) and mfcc.ndim == 2 and mfcc.shape[1] == 13:
                    self.X_all.append(mfcc)
                    self.y_all.append(label)

        self.indices = np.arange(len(self.X_all))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.X_all) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        X_batch = []
        y_batch = []
        for i in batch_indices:
            mfcc = self.X_all[i]
            padded = tf.keras.preprocessing.sequence.pad_sequences(
                [mfcc], maxlen=self.max_len, padding='post', dtype='float32'
            )[0]
            X_batch.append(padded)
            y_batch.append(self.y_all[i])

        X_batch = np.array(X_batch).reshape(-1, 157, 13, 1)
        y_batch = np.array(y_batch)
        return X_batch, y_batch

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

train_generator = MFCCBatchGenerator(mfcc_paths, label_paths, batch_size=16)
print("✅ 유효한 학습용 샘플 수:", len(train_generator) * 16)

# 5. 모델 정의
from tensorflow.keras import models, layers

def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.GlobalAveragePooling2D())

    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# 6. 모델 생성 및 콜백 정의
input_shape = (157, 13, 1)
num_classes = 8
model = create_cnn_model(input_shape, num_classes)

checkpoint_path = "/media/usou/PortableSSD/mldl_project/models/best_model_generator.h5"
callbacks = [
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
]

# 7. 가비지 컬렉션
gc.collect()

# 8. 학습 시작
history = model.fit(
    train_generator,
    validation_data=(X_val, y_val),
    epochs=30,
    callbacks=callbacks
)


# 저장된 모델 불러오기


In [ ]:
from tensorflow.keras.models import load_model

model_path = "/media/usou/PortableSSD/mldl_project/models/best_model_generator.h5"
model = load_model(model_path)
print("✅ 모델 로드 완료")


In [ ]:
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

acc = accuracy_score(y_val, y_pred_classes)
print(f"✅ 검증 정확도: {acc:.4f}\n")

print("📊 분류 리포트:")
print(classification_report(y_val, y_pred_classes))

print("🌀 혼동 행렬:")
print(confusion_matrix(y_val, y_pred_classes))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

cm = confusion_matrix(y_val, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()


# 학습 로그 저장

In [ ]:
import pickle

# 학습 직후 저장
with open("/media/usou/PortableSSD/mldl_project/models/history.pkl", "wb") as f:
    pickle.dump(history.history, f)

print("✅ 학습 로그 저장 완료")


# 학습 로그 불러 오기

In [ ]:
import pickle

with open("/media/usou/PortableSSD/mldl_project/models/history.pkl", "rb") as f:
    history_dict = pickle.load(f)

print("✅ 학습 로그 불러오기 완료")


# 시각화

In [ ]:
import matplotlib.pyplot as plt

acc = history_dict["accuracy"]
val_acc = history_dict["val_accuracy"]
loss = history_dict["loss"]
val_loss = history_dict["val_loss"]
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


# epoch 300 으로 accuracy 더 올리기

In [ ]:
# 1. 기본 라이브러리 불러오기 및 GPU 설정
import numpy as np
import tensorflow as tf
import os, gc
from glob import glob
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# GPU 메모리 과도할당 방지 설정
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# 2. MFCC 및 레이블 파일 경로 설정
mfcc_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches"
label_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches/encoded_labels"
mfcc_paths = sorted(glob(os.path.join(mfcc_dir, "mfcc_batch_*.npy")))
label_paths = sorted(glob(os.path.join(label_dir, "label_batch_*.npy")))

# 3. 검증 데이터셋 구성 (112157개 샘플)
val_sample_size = 112157
X_val_list, y_val_list = [], []
loaded = 0

for mfcc_path, label_path in zip(reversed(mfcc_paths), reversed(label_paths)):
    mfcc_batch = np.load(mfcc_path, allow_pickle=True)
    label_batch = np.load(label_path)
    
    for mfcc, label in zip(mfcc_batch, label_batch):
        if isinstance(mfcc, np.ndarray) and mfcc.ndim == 2 and mfcc.shape[1] == 13:
            padded = pad_sequences([mfcc], maxlen=157, padding='post', dtype='float32')[0]
            X_val_list.append(padded)
            y_val_list.append(label)
            loaded += 1
            if loaded >= val_sample_size:
                break
    if loaded >= val_sample_size:
        break

X_val = np.array(X_val_list).reshape(-1, 157, 13, 1)
y_val = np.array(y_val_list)

print("검증 데이터 준비 완료")
print("검증용 X shape:", X_val.shape)
print("검증용 y shape:", y_val.shape)

# 4. 학습용 데이터 제너레이터 정의
class MFCCBatchGenerator(tf.keras.utils.Sequence):
    def __init__(self, mfcc_paths, label_paths, batch_size=16, max_len=157, shuffle=True):
        self.max_len = max_len
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.X_all = []
        self.y_all = []

        for mfcc_path, label_path in zip(mfcc_paths, label_paths):
            mfcc_batch = np.load(mfcc_path, allow_pickle=True)
            label_batch = np.load(label_path)
            for mfcc, label in zip(mfcc_batch, label_batch):
                if isinstance(mfcc, np.ndarray) and mfcc.ndim == 2 and mfcc.shape[1] == 13:
                    self.X_all.append(mfcc)
                    self.y_all.append(label)

        self.indices = np.arange(len(self.X_all))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.X_all) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        X_batch, y_batch = [], []
        for i in batch_indices:
            mfcc = self.X_all[i]
            padded = pad_sequences([mfcc], maxlen=self.max_len, padding='post', dtype='float32')[0]
            X_batch.append(padded)
            y_batch.append(self.y_all[i])
        X_batch = np.array(X_batch).reshape(-1, 157, 13, 1)
        y_batch = np.array(y_batch)
        return X_batch, y_batch

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

train_generator = MFCCBatchGenerator(mfcc_paths, label_paths, batch_size=16)
print("유효한 학습용 샘플 수:", len(train_generator) * 16)

# 5. CNN 모델 정의 함수
from tensorflow.keras import models, layers

def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.GlobalAveragePooling2D())

    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# 6. 모델 구성 및 콜백 정의
input_shape = (157, 13, 1)
num_classes = 8
model = create_cnn_model(input_shape, num_classes)

checkpoint_path = "/media/usou/PortableSSD/mldl_project/models/best_model_generator.h5"
callbacks = [
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=30, restore_best_weights=True)
]

# 7. 불필요한 메모리 수거
gc.collect()

# 8. 모델 학습 (epoch 300으로 확장)
history = model.fit(
    train_generator,
    validation_data=(X_val, y_val),
    epochs=300,
    callbacks=callbacks
)


# 모델 저장 및 시각화

In [ ]:
import json
import matplotlib.pyplot as plt

# 로그 저장 경로
log_path = "/media/usou/PortableSSD/mldl_project/models/train_history_log.json"

# history 객체를 JSON으로 저장
with open(log_path, "w") as f:
    json.dump(history.history, f)
print("✅ 학습 로그 저장 완료")

# 시각화
plt.figure(figsize=(12, 5))

# 정확도 그래프
plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.title("Model Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

# 손실 그래프
plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.title("Model Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.tight_layout()
plt.show()


# .keras 포맷 으로 저장

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("/media/usou/PortableSSD/mldl_project/models/best_model_generator.h5")


In [ ]:
model.save("best_model_generator.keras")

# 모델 테스트

## m4a → wav 변환

In [ ]:
from pydub import AudioSegment
import os

def convert_to_wav(input_path, output_path=None):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"파일이 존재하지 않아요: {input_path}")

    if output_path is None:
        output_path = os.path.splitext(input_path)[0] + ".wav"

    audio = AudioSegment.from_file(input_path, format="m4a")
    audio.export(output_path, format="wav")
    print("✅ 변환 완료:", output_path)
    return output_path


## m4a, mp3, mp4 -> wav

In [13]:
import os
from pydub import AudioSegment

def convert_to_wav(input_path, output_path=None):
    """
    오디오 또는 비디오 파일을 WAV로 변환합니다.
    지원 포맷: .wav, .m4a, .mp3, .mp4 등
    내부적으로 ffmpeg가 설치되어 있어야 합니다.

    :param input_path: 입력 파일 경로
    :param output_path: 출력 파일 경로 (None이면 자동 생성)
    :return: 변환된 WAV 파일 경로
    """
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"파일이 존재하지 않아요: {input_path}")

    ext = os.path.splitext(input_path)[1].lower()
    if output_path is None:
        output_path = os.path.splitext(input_path)[0] + ".wav"

    # pydub이 ffmpeg를 통해 모든 형식 처리함
    try:
        audio = AudioSegment.from_file(input_path)
        audio.export(output_path, format="wav")
    except Exception as e:
        raise RuntimeError(f"오디오 변환 실패: {e}")

    print("변환 완료:", output_path)
    return output_path


## MFCC 추출(모델 입력 준비)

In [14]:
import librosa
import numpy as np

def extract_mfcc(file_path, sr=16000, n_mfcc=40):
    y, sr = librosa.load(file_path, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc = np.expand_dims(mfcc, axis=-1)  # CNN 입력을 위해 채널 차원 추가
    return mfcc


## 레이블 인코딩

In [15]:
import os
import numpy as np
import glob
import pickle
from sklearn.preprocessing import LabelEncoder

# ============================
# 1. 설정
# ============================
# 레이블 배치가 저장된 경로
label_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches"

# 인코딩된 레이블 저장 경로
encoded_label_dir = os.path.join(label_dir, "encoded_labels")
os.makedirs(encoded_label_dir, exist_ok=True)

# ============================
# 2. 모든 배치 레이블 수집 및 'nan' 제거
# ============================
label_files = sorted(glob.glob(os.path.join(label_dir, "label_batch_*.npy")))

all_labels = []
batch_label_data = []
valid_indices_per_batch = []

for label_file in label_files:
    labels = np.load(label_file, allow_pickle=True)
    
    # 'nan' 문자열 제거
    valid_indices = [i for i, l in enumerate(labels) if str(l).lower() != 'nan']
    valid_labels = [labels[i] for i in valid_indices]

    all_labels.extend(valid_labels)
    batch_label_data.append(valid_labels)
    valid_indices_per_batch.append(valid_indices)

# ============================
# 3. 레이블 인코딩
# ============================
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

# 인코더 저장
with open(os.path.join(label_dir, "label_encoder.pkl"), "wb") as f:
    pickle.dump(label_encoder, f)

# ============================
# 4. 인코딩된 레이블 배치별로 저장
# ============================
for i, labels in enumerate(batch_label_data):
    encoded = label_encoder.transform(labels)
    save_path = os.path.join(encoded_label_dir, f"label_batch_{i}.npy")
    np.save(save_path, encoded)

print(f"총 유효 레이블 개수 (nan 제외): {len(all_labels)}")
print(f"인코딩된 클래스 목록: {label_encoder.classes_}")
print(f"유효 배치 수: {len(label_files)}")
print("nan 제거 및 레이블 인코딩 완료")


총 유효 레이블 개수 (nan 제외): 530280
인코딩된 클래스 목록: ['Angry' 'Anxious' 'Embarrassed' 'Happy' 'Hurt' 'Neutrality' 'Sad']
유효 배치 수: 82
nan 제거 및 레이블 인코딩 완료


## 감정 분석 예측

In [16]:
from tensorflow.keras.models import load_model
import numpy as np

# 모델 로드
model_path = "best_model_generator_voice_emotion_analyze.keras"
model = load_model(model_path)

# LabelEncoder 로드
import pickle
with open("label_encoder.pkl", "rb") as f:
    le = pickle.load(f)

def predict_emotion(mfcc_data):
    # 입력 형태 맞추기 (1, time, freq, 1)
    X = np.expand_dims(mfcc_data, axis=0)
    pred = model.predict(X)
    idx = np.argmax(pred)
    label = le.inverse_transform([idx])[0]
    confidence = float(np.max(pred))
    return label, confidence


/home/usou/venv/test_super/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 34 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


## 전체 파이프라인 실행 예시

In [17]:
# 1. 파일 변환
m4a_path = "/media/usou/PortableSSD/mldl/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/VS1/1.감정/5.상처/0005_G1A3E5S0C0_LJB/0005_G1A3E5S0C0_LJB_000011.wav"

wav_path = convert_to_wav(m4a_path)

# 2. MFCC 추출
mfcc = extract_mfcc(wav_path)

# 3. 감정 예측
emotion, score = predict_emotion(mfcc)
print(f"\n 감정 분석 결과: {emotion} ({score:.2%} 확신)")


변환 완료: /media/usou/PortableSSD/mldl/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/VS1/1.감정/5.상처/0005_G1A3E5S0C0_LJB/0005_G1A3E5S0C0_LJB_000011.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 412ms/step

 감정 분석 결과: Happy (100.00% 확신)


In [10]:
import librosa
import numpy as np
import tensorflow as tf
import pickle

# 설정
wav_path = "/media/usou/PortableSSD/mldl/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/VS1/1.감정/5.상처/0005_G1A3E5S0C0_LJB/0005_G1A3E5S0C0_LJB_000011.wav"  # 예: ./val_data/sample_001.wav
label_encoder_path = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches/label_encoder.pkl"  # 예: ./label_encoder.pkl
model_path = "best_model_generator_voice_emotion_analyze.keras"


# 1. 모델 및 라벨 인코더 로드
model = tf.keras.models.load_model(model_path)

with open(label_encoder_path, "rb") as f:
    label_encoder = pickle.load(f)

# 2. wav 파일에서 MFCC 추출
def extract_mfcc(wav_path, sr=16000, n_mfcc=40, max_len=300):
    y, sr = librosa.load(wav_path, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    
    # 길이 고정 (짧으면 패딩, 길면 자름)
    if mfcc.shape[1] < max_len:
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]

    return mfcc

# 3. 예측
mfcc = extract_mfcc(wav_path)
mfcc_input = mfcc[np.newaxis, ..., np.newaxis]  # (1, 40, 300, 1) 형태로 reshape

y_pred = model.predict(mfcc_input)

# 4. 결과 출력
pred_index = np.argmax(y_pred)
pred_emotion = label_encoder.inverse_transform([pred_index])[0]

print("Softmax 확률 분포:", y_pred[0])
print("예측된 감정:", pred_emotion)


/home/usou/venv/test_super/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 34 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
2025-04-02 12:21:20.363584: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{k25=2} for conv %cudnn-conv-bias-activation.11 = (f32[1,128,10,75]{3,2,1,0}, u8[0]{0}) custom-call(f32[1,64,10,75]{3,2,1,0} %bitcast.440, f32[128,64,3,3]{3,2,1,0} %bitcast.447, f32[128]{0} %bitcast.449), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_1/conv2d_2_1/convolution" source_file="/home/usou/venv/test_super/lib/python3.12/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_o

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 694ms/step
Softmax 확률 분포: [6.35206882e-17 7.89071120e-13 1.22968635e-10 1.00000000e+00
 3.78825990e-27 1.18888200e-12 5.54563053e-27 1.00396704e-16]
예측된 감정: Happy


## 재학습

-  감정 클래스 7 -> 4개 축소
- 행복, 슬픔, 분노, 중립
- 데이터 증강 및 개선 모델 필요
- 모델 성능 향상 전략
    - 데이터 증강 + 하이퍼 파라미터 튜닝 + 과적합 방지

# 전체 흐름 요약
✅ 전체 흐름 요약

| 단계         | 설명                                              |
|--------------|---------------------------------------------------|
| 1단계 (완료) | JSON → `metadata_cleaned.csv` 생성               |
| 2단계        | 4개 감정만 필터링 → `metadata_4class.csv` 저장   |
| 3단계        | MFCC 추출 + 증강 적용                             |
| 4단계        | 인코딩 → 배치 분할 저장                          |


# 1단계: JSON → DataFrame 변환 

In [6]:
import os
import json
import pandas as pd

# 라벨링 JSON이 위치한 최상위 폴더 경로
label_root = "/media/usou/PortableSSD/mldl/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training/라벨링데이터/"

# 실제 wav 파일이 존재하는 원천 데이터 경로
wav_root = "/media/usou/PortableSSD/mldl/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training/원천데이터/"

# 결과 CSV 및 로그를 저장할 폴더 경로
save_dir = "./data/usou"
os.makedirs(save_dir, exist_ok=True)

# 사용할 감정 4개만 필터링 대상
target_classes = ['Happy', 'Sad', 'Angry', 'Neutrality']

# 정제된 데이터와 누락 파일 로그를 담을 리스트
data = []
broken_files = []

# 모든 하위 폴더를 순회하며 JSON 파일 탐색
for folder_path, _, files in os.walk(label_root):
    for file_name in files:
        if not file_name.endswith(".json"):
            continue  # JSON 파일만 처리

        json_path = os.path.join(folder_path, file_name)
        try:
            # JSON 파일 로딩
            with open(json_path, 'r', encoding='utf-8') as f:
                content = json.load(f)

            # 감정 정보 추출 및 필터링
            emotion = content["화자정보"]["Emotion"]
            if emotion not in target_classes:
                continue  # 지정된 감정이 아닌 경우 제외

            # 그 외 부가 정보 추출
            style = content["화자정보"].get("SpeechStyle", "N/A")
            sensitivity = content["화자정보"].get("Sensitivity", "N/A")
            wav_file = content["파일정보"]["FileName"]

            # 경로 변환 (TL → TS로 수정하여 wav 경로 재구성)
            relative_path = os.path.relpath(folder_path, start=label_root)
            relative_path = relative_path.replace("TL", "TS")
            wav_path = os.path.join(wav_root, relative_path, wav_file)

            # wav 파일 존재 여부 확인 후 저장
            if os.path.exists(wav_path):
                data.append({
                    "wav_path": wav_path,
                    "emotion": emotion,
                    "style": style,
                    "sensitivity": sensitivity
                })
            else:
                broken_files.append(wav_path)
        except Exception as e:
            # JSON 로딩 실패 또는 파싱 오류 발생 시 경로 저장
            broken_files.append(json_path)

# 수집된 데이터를 DataFrame으로 변환
df = pd.DataFrame(data)

# 결과 CSV 저장
df.to_csv(os.path.join(save_dir, "metadata_4class.csv"), index=False)

# 누락된 경로 저장
with open(os.path.join(save_dir, "broken_files.txt"), "w") as f:
    for path in broken_files:
        f.write(path + "\n")

# 최종 처리 결과 출력
print("총 수집된 샘플:", len(df))
print("에러 발생 수:", len(broken_files))


총 수집된 샘플: 300584
에러 발생 수: 0


# MFCC 추출 + 증강

- metadata_4class.csv에서 경로와 감정 읽기

- audiomentations로 증강 (2배)

- MFCC 추출 (40 x 300)

- mfcc_data.npy, labels.npy 저장

In [1]:
import os
import numpy as np
import pandas as pd
import librosa
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift
from tqdm import tqdm

# 1. 입력 및 출력 경로 설정
csv_path = "./data/usou/metadata_4class.csv"
output_dir = "/media/usou/PortableSSD/mldl_project/data4class_batches"
os.makedirs(output_dir, exist_ok=True)

# 2. 오디오 및 MFCC 관련 설정
n_mfcc = 40                     # 추출할 MFCC 계수 수
max_len = 300                  # MFCC 길이 고정값
n_augment = 2                  # 증강 횟수 (원본 포함 총 3배)
batch_size = 1000              # 배치 저장 단위

# 3. 데이터프레임 불러오기
df = pd.read_csv(csv_path)

# 4. 증강 파이프라인 설정
augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.7),
    TimeStretch(min_rate=0.9, max_rate=1.1, p=0.5),
    PitchShift(min_semitones=-2, max_semitones=2, p=0.5),
    Shift(min_shift=-0.2, max_shift=0.2, p=0.5)
])

# 5. MFCC 추출 함수 정의
def extract_mfcc(y, sr):
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    if mfcc.shape[1] < max_len:
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    return mfcc

# 6. 배치별로 데이터를 저장하기 위한 변수 초기화
mfcc_batch = []
label_batch = []
batch_index = 0

# 7. tqdm을 이용해 전체 진행 상황 표시
for idx, row in tqdm(df.iterrows(), total=len(df), desc="MFCC 추출 및 증강"):
    wav_path = row["wav_path"]
    emotion = row["emotion"]

    try:
        # 원본 로딩
        y, sr = librosa.load(wav_path, sr=16000)

        # 원본 MFCC 저장
        mfcc_batch.append(extract_mfcc(y, sr))
        label_batch.append(emotion)

        # 증강 MFCC 저장
        for _ in range(n_augment):
            y_aug = augment(samples=y, sample_rate=sr)
            mfcc_batch.append(extract_mfcc(y_aug, sr))
            label_batch.append(emotion)

        # 일정 크기의 배치가 모이면 저장 후 초기화
        if len(mfcc_batch) >= batch_size:
            np.save(os.path.join(output_dir, f"mfcc_batch_{batch_index}.npy"), np.array(mfcc_batch))
            np.save(os.path.join(output_dir, f"label_batch_{batch_index}.npy"), np.array(label_batch, dtype=object))
            print(f"배치 {batch_index} 저장 완료 - {len(mfcc_batch)}개")
            batch_index += 1
            mfcc_batch.clear()
            label_batch.clear()

    except Exception as e:
        print(f"오류 발생: {wav_path} → {e}")
        continue

# 8. 마지막에 남은 배치도 저장
if mfcc_batch:
    np.save(os.path.join(output_dir, f"mfcc_batch_{batch_index}.npy"), np.array(mfcc_batch))
    np.save(os.path.join(output_dir, f"label_batch_{batch_index}.npy"), np.array(label_batch, dtype=object))
    print(f"마지막 배치 {batch_index} 저장 완료 - {len(mfcc_batch)}개")

print("전체 MFCC 추출 및 저장 작업 완료")


/tmp/ipykernel_4643/3246010131.py:20: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)
MFCC 추출 및 증강:   0%|          | 335/300584 [00:25<7:26:13, 11.21it/s]

배치 0 저장 완료 - 1002개


MFCC 추출 및 증강:   0%|          | 669/300584 [00:52<7:05:35, 11.75it/s]

배치 1 저장 완료 - 1002개


MFCC 추출 및 증강:   0%|          | 1003/300584 [01:19<6:33:32, 12.69it/s]

...

배치 897 저장 완료 - 1002개


MFCC 추출 및 증강: 100%|█████████▉| 300268/300584 [6:09:19<00:24, 12.92it/s]

배치 898 저장 완료 - 1002개


MFCC 추출 및 증강: 100%|██████████| 300584/300584 [6:09:40<00:00, 13.55it/s]

마지막 배치 899 저장 완료 - 954개
전체 MFCC 추출 및 저장 작업 완료


# Validation

## JSON -> DataFrame

In [2]:
import os
import json
import pandas as pd

# 검증 데이터의 JSON 폴더 경로 (변경 필요)
label_root = "/media/usou/PortableSSD/mldl/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training/라벨링데이터/"
wav_root   = "/media/usou/PortableSSD/mldl/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training/원천데이터/"

save_dir = "./data/usou"
os.makedirs(save_dir, exist_ok=True)

target_classes = ['Happy', 'Sad', 'Angry', 'Neutrality']

data = []
broken_files = []

for folder_path, _, files in os.walk(label_root):
    for file_name in files:
        if not file_name.endswith(".json"):
            continue
        json_path = os.path.join(folder_path, file_name)
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                content = json.load(f)

            emotion = content["화자정보"]["Emotion"]
            if emotion not in target_classes:
                continue

            style = content["화자정보"].get("SpeechStyle", "N/A")
            sensitivity = content["화자정보"].get("Sensitivity", "N/A")
            wav_file = content["파일정보"]["FileName"]

            relative_path = os.path.relpath(folder_path, start=label_root)
            relative_path = relative_path.replace("TL", "TS")
            wav_path = os.path.join(wav_root, relative_path, wav_file)

            if os.path.exists(wav_path):
                data.append({
                    "wav_path": wav_path,
                    "emotion": emotion,
                    "style": style,
                    "sensitivity": sensitivity
                })
            else:
                broken_files.append(wav_path)
        except Exception as e:
            broken_files.append(json_path)

df = pd.DataFrame(data)
df.to_csv(os.path.join(save_dir, "metadata_4class_val.csv"), index=False)

with open(os.path.join(save_dir, "broken_files_val.txt"), "w") as f:
    for path in broken_files:
        f.write(path + "\n")

print("총 수집된 검증 샘플 수:", len(df))
print("에러 발생 수:", len(broken_files))


총 수집된 검증 샘플 수: 300584
에러 발생 수: 0


# 검증용 MFCC 추출

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm

# 1. 검증용 metadata 경로 및 결과 저장 폴더 설정
csv_path = "./data/usou/metadata_4class_val.csv"
output_dir = "/media/usou/PortableSSD/mldl_project/data4class_val_batches"
os.makedirs(output_dir, exist_ok=True)

# 2. MFCC 관련 설정
n_mfcc = 40
max_len = 300
batch_size = 1000  # 배치 저장 단위

# 3. CSV 파일 로드
df = pd.read_csv(csv_path)

# 4. MFCC 추출 함수 정의
def extract_mfcc(y, sr):
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    if mfcc.shape[1] < max_len:
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    return mfcc

# 5. 배치 저장 준비
mfcc_batch = []
label_batch = []
batch_index = 0

# 6. tqdm을 이용한 처리 루프
for _, row in tqdm(df.iterrows(), total=len(df), desc="검증용 MFCC 배치 추출"):
    wav_path = row["wav_path"]
    emotion = row["emotion"]

    try:
        y, sr = librosa.load(wav_path, sr=16000)
        mfcc = extract_mfcc(y, sr)
        mfcc_batch.append(mfcc)
        label_batch.append(emotion)

        # 배치 크기 도달 시 저장
        if len(mfcc_batch) >= batch_size:
            np.save(os.path.join(output_dir, f"mfcc_val_batch_{batch_index}.npy"), np.array(mfcc_batch))
            np.save(os.path.join(output_dir, f"label_val_batch_{batch_index}.npy"), np.array(label_batch, dtype=object))
            print(f"배치 {batch_index} 저장 완료 - {len(mfcc_batch)}개")
            batch_index += 1
            mfcc_batch.clear()
            label_batch.clear()

    except Exception as e:
        print(f"오류 발생: {wav_path} → {e}")
        continue

# 7. 마지막 배치 저장
if mfcc_batch:
    np.save(os.path.join(output_dir, f"mfcc_val_batch_{batch_index}.npy"), np.array(mfcc_batch))
    np.save(os.path.join(output_dir, f"label_val_batch_{batch_index}.npy"), np.array(label_batch, dtype=object))
    print(f"마지막 배치 {batch_index} 저장 완료 - {len(mfcc_batch)}개")

print("검증용 MFCC 배치 저장 완료")
